In [1]:
import pandas as pd 
import numpy as np 
#from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
#from mlxtend.preprocessing import TransactionEncoder
from efficient_apriori import apriori

In [2]:
trans = pd.read_csv("transactions.csv", sep = "|")
book =  pd.read_csv("book_data_extended.csv", sep = "|")

In [3]:
book.head()

,Unnamed: 0,itemID,title,author,publisher,main topic,subtopics,gender_author,title_lang
0,0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],female,en
1,1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]",female,de
2,2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]",female,en
3,3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]",NaN,de
4,4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]",female,de


In [4]:
trans.head

<bound method NDFrame.head of         sessionID  itemID  click  basket  order
0               0   21310      1       0      0
1               1   73018      1       0      0
2               2   19194      1       0      0
3               3   40250      1       0      0
4               4   46107      1       0      0
...           ...     ...    ...     ...    ...
365138     279351   70183      1       0      0
365139     279352   39716      1       0      0
365140     279353   35260      1       0      0
365141     279353   18805      4       0      0
365142     279354    3083      1       0      0

[365143 rows x 5 columns]>

In [ ]:
sum([x in set(trans.itemID) for x in set(book.itemID)]) / len(list(set(book.itemID)))

In [19]:
np.mean(trans['sessionID'].value_counts())

1.3425214075879743

In [44]:
 np.mean([len(list(set(trans[trans.sessionID == x]["itemID"]))) for x in set(trans.sessionID) ] )

1.3425214075879743

In [ ]:
trans_at_least2 = trans[trans['sessionID'].isin(trans['sessionID'].value_counts()[trans['sessionID'].value_counts()>1].index)]

In [13]:
print(len(list(set(trans_at_least2["sessionID"]))))

36341


In [42]:
len(set(trans_at_least2["itemID"]))

14471

In [14]:
print(len(list(set(trans_at_least2["itemID"]))))

14471


In [5]:
def learnrules(book = book, trans = trans, attributes = ["subtopics", "author","main topic","title","itemID","publisher","title_lang"]):

# Limit data to only transacations with at least two items
    trans_at_least2 = trans[trans['sessionID'].isin(trans['sessionID'].value_counts()[trans['sessionID'].value_counts()>1].index)]

    unique_sessionids = list(set(trans_at_least2["sessionID"]))
    model_input_lists = [None] * len(unique_sessionids)
    learned_rules = [None] * len(attributes) 

    # Prepare Date for a priori algo. 
    for i in range(len(attributes)):
        result_counter = 0
        attribute_to_prepare = attributes[i]

        # substitute new attribute in for itemID
        if attribute_to_prepare != "itemID":
            substitution_dict = dict(zip(book.itemID, book[attribute_to_prepare]))
            substitued_trans = trans_at_least2.copy(deep=True)
            substitued_trans.itemID = substitued_trans.itemID.map(substitution_dict)
        else:
            substitued_trans = trans_at_least2


        # turn transaction into lists of touples of varying length
        model_input_lists = [None] * len(unique_sessionids) 

        if attribute_to_prepare != "subtopics":
            for j in range(len(unique_sessionids)):
                subset = substitued_trans[substitued_trans['sessionID'] == unique_sessionids[j]]
                model_input_lists[j] = list(subset.itemID.astype(str))
        else: 
            for j in range(len(unique_sessionids)): 
                subset = substitued_trans[substitued_trans['sessionID'] == unique_sessionids[j]]
                result_set = set()
                for x in subset.itemID:
                    [result_set.add(y) for y in x.replace("]","").replace("[","").split(",")]
                model_input_lists[j] = list(result_set)

        for k in range(len(model_input_lists)):
            helper_set = set()
            touple_ = model_input_lists[k]
            for l in range(len(touple_)):
                element = touple_[l]
                # to get same same relations add ___ to every second occurence of an attribute in a touple
                if element in helper_set:
                    model_input_lists[k][l] = element +  "___" 
                else:
                    helper_set.add(element)
        model_input_lists = [ tuple(x) for x in model_input_lists ]

        # run rule learner
        itemsets, rules = apriori(model_input_lists, min_confidence = 0.9, min_support=0.001,max_length = 2)
        
        # remove ___ again from learned rules
        for m in range(len(rules)):
            rule = rules[m]
            rule.lhs = list(rule.lhs)[0].replace("___","")  
            rule.rhs = list(rule.rhs)[0].replace("___","")  
            rules[m] = rule

        learned_rules[i] = rules
    return(learned_rules)

In [7]:
rule_set = learnrules(book = book, trans = trans, attributes = ["subtopics", "author","main topic","title","itemID","publisher","title_lang"])

In [8]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [55]:
def recommend(rule_set = rule_set,booke = book, to_recommend = 40250, attribute_weights = {"subtopics":0.5, "author":1,"main topic":2,"title":5,"itemID":0,"publisher":2,"title_lang":0.0}):
    # has to be specified in the same oder as above!!!!
    to_recommend_for = book[book["itemID"] == to_recommend]

    scores_for_each_book = np.zeros(shape = (len(attribute_weights.keys()),len(book) ) )

    for i in range(len(attribute_weights.keys())):
        attribute = list(attribute_weights.keys())[i]
        if attribute != "subtopics":
            # Extract value from case for which we should recommend
            attribute_value = to_recommend_for[attribute].tolist()[0]

            # Select applicable rules
            attribute_rules = [x for x in rule_set[i] if  str(attribute_value) in str(x.lhs)]

            # Select antecedents from rules
            recommend_if = [x.rhs for x in attribute_rules]

            # Flatten for easier useage
            recommend_if = flatten_list([ list(x) for x in recommend_if])

            # Calculate scores for each possible book
            scores_for_each_book[i] = np.where( [x in recommend_if for x in book[attribute]], attribute_weights[attribute], 0)

        else:
            # Extract value from case for which we should recommend
            attribute_values = str(to_recommend_for[attribute].tolist()[0]).replace("]","").replace("[","").split(",")
            zwischensumme = np.zeros(shape = (len(attribute_values),len(book)))
            for j in range(len(attribute_values)):
                attribute_value = attribute_values[j]
                attribute_rules = [x for x in rule_set[i] if  attribute_value in x.lhs]
                recommend_if = [x.rhs for x in attribute_rules]
                recommend_if = flatten_list([ list(x) for x in recommend_if])
                listed_subtopics =  [str(x).replace("]","").replace("[","").split(",") for x in book[attribute].tolist() ]
                zwischensumme[i] = np.where( [not set(recommend_if).isdisjoint(set(x)) for x in listed_subtopics]  ,attribute_weights[attribute], 0)
            scores_for_each_book[i] =  np.sum(zwischensumme,axis=0)

    books_recomend = book.copy(deep = True)
    scores = np.sum(scores_for_each_book,axis=0)
    books_recomend["recommendation_scores"] = scores
    books_recomend_sorted = books_recomend.sort_values(by='recommendation_scores', ascending = False)
    books_recomend_sorted = books_recomend_sorted[books_recomend_sorted['title_lang'] == to_recommend_for["title_lang"].tolist()[0] ]
    recommendations = books_recomend_sorted.head(5).itemID
    return(recommendations)
    #return(recommendations, books_recomend_sorted)

In [75]:
data = pd.read_csv("evaluation.csv")
results = {}
ids = list(data["itemID"])
ids.remove(37378)
ids.remove(47675)
for item in ids:
    results[item] =  recommend(rule_set = rule_set,booke = book, to_recommend = item, attribute_weights = {"subtopics":0.5, "author":1,"main topic":2,"title":5,"itemID":0,"publisher":2,"title_lang":0.0})

12
45274
10104
41371
14015
28515
70766
1822
56794
60188
62060
31401
6420
72855
73378
25049
24603
5737
38980
13382
75121
39779
14026
9392
77956
70300
77134
75340
51066
67776
53929
22342
41116
60331
65307
50039
42184
76163
2319
62494
31015
21497
53008
39308
4690
19777
10078
65353
58847
61593
18537
73427
7521
71586
13772
33092
74398
74089
57282
38760
15657
17665
75251
56282
73973
19994
47098
54190
61043
78675
40394
43844
27300
22870
61707
17888
48292
2054
6643
3857
64350
10833
33556
23256
53880
75694
17770
9365
11177
48659
18169
48490
70627
40000
18665
35574
11732
6798
42415
64236
60644
11595
39957
1519
10075
25209
7108
67848
40424
72523
60588
21315
50356
75904
17240
24218
21949
6478
76242
71112
76437
52656
10304
25399
71619
31439
14323
10842
39515
59732
56452
53496
27903
53672
29813
67909
36620
44197
68772
72058
17921
67832
73203
30873
46059
49615
76629
55488
30713
3527
52884
48270
18224
39309
11265
8288
40312
35418
78181
8859
3875
17348
37321
22277
52705
26856
36095
66497
55326
3959
669

In [99]:
out_file = open("Resultfile.json", "w")
json.dump(ids, out_file, indent = 6)
out_file.close()